In [1]:
import torch
import torch.utils.data as data_utils
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
import time
import copy
from torch.optim import lr_scheduler
from torchvision import datasets
from sklearn.metrics import roc_curve

max_epochs=3

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [4]:
model = models.vgg19(pretrained=False)

for param in model.parameters():
    param.requires_grad = False
    
in_features = model.classifier[-1].in_features

model.classifier[6] = nn.Linear(in_features, 2)                   

criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1,5]).to(device))
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = model.cuda()

In [16]:
malign_count = 0
for chunk in range(10):
    Y = torch.load('data/labels-' + str(chunk) + '.pt')
    print(Y.sum())
    malign_count += Y.sum()
1 - 2286/21639

tensor(228)
tensor(216)
tensor(229)
tensor(236)
tensor(239)
tensor(225)
tensor(244)
tensor(226)
tensor(226)
tensor(217)


0.8943574102315264

In [8]:
train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10, chunks=9)

Epoch 1/10
----------
train Loss: 101.7711 Acc: 0.4917
val Loss: 11.8745 Acc: 0.7980
Epoch 2/10
----------
train Loss: 81.2141 Acc: 0.5205
val Loss: 14.6899 Acc: 0.8307
Epoch 3/10
----------
tensor([ 21.7659,  67.3232,  31.6996, 154.5877,  31.2627, 152.3798,  84.2595,
        187.6634, 126.5163, 123.3075,  44.4879,  31.0935,  -4.6674,  92.8584,
         45.8597,  67.7417,  98.6124, 162.7464, 119.4016,   4.4486,  32.2928,
          3.4130,  33.8120,  88.9051, 168.2034,  32.9952, 215.4153,  55.4850,
        109.3170,  14.2181, 211.2370, 145.8534,  64.9913, 140.6700,   0.4803,
         37.2375,  49.2092,  37.9319,  73.5203,  74.4357, 157.9990,   1.4631,
         18.6665,  86.8316, 128.5952,  70.7716,  93.1061,   8.1524,  66.7859,
         63.6580,  11.7171,  69.9743, 176.9747,  35.4362,  96.1671, 178.6960,
         72.8020, 184.3669, 144.9571,  12.7641,  63.4026,  22.5484,  83.8251,
         20.5920], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 

tensor([138.8690,  88.0592,   5.3088,  78.6778, 185.5705,  19.1102,  51.5677,
         32.4819,  74.0085, 232.1406,  -2.0014,  62.5720, 113.0206,  87.7438,
        102.7514, 130.8787, 102.2491,  43.6750, 139.1896, 168.9995,  62.7660,
         95.5045, 207.3048, 307.1244,  24.8291,  11.1231,  54.6141,  35.9299,
         34.4741,  85.9331, 176.4812,  10.9283,  16.8201, 151.4515,  91.8220,
        104.0250,  51.8913,  99.6804,  19.4991,  71.7598,   7.1361,   4.4602,
         78.2253, 258.3453,  61.1125,  23.3094,  10.0633,  63.8445,  57.0463,
         47.0489,   9.1632, 148.5978, 162.1703, 256.9247,  43.1474,  13.9384,
        160.9823, 136.8916,  81.4113, 146.1993, 148.6964, 119.8320, 122.5605,
        124.2807], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0], device='cuda:0')
tens

tensor([ 64.9603, 118.9478,  81.2990,  69.8727,  79.8198,  73.6535, 140.7253,
         89.9593,  94.0508,   6.1331, 191.3885, 170.0962, 147.2737,  88.5879,
          9.6175,  59.6864,  40.2850,  19.5916, 200.4661, 159.5413,  23.9716,
         56.4343,   7.1105, 154.7922,  69.8426,   7.5645,  50.7371, 223.6132,
         44.2085, 138.6529, 154.2694, 115.5238,  13.9973,  82.6985,  80.3973,
        101.7344,  10.4172,  37.5453,  57.7581,  57.5570,  35.5392,  37.9458,
         -0.5707,  86.7672,  55.9209,  99.8850, 142.2278,  68.1004, 104.7295,
        103.2901,  70.1913,  39.3659,  29.9837, 265.6473, 220.7770, 191.3312,
         93.2081,   7.5445, 182.9876,  37.0767,  92.6267,  78.6896,   2.1822,
        120.7408], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], device='cuda:0')
tens

tensor([ 2.5239e+02,  1.9736e+02,  9.3261e+01,  9.2832e+01,  2.1701e+01,
         1.0925e+02,  4.4678e+01,  6.5951e+01,  9.2942e+01,  1.8834e+01,
         4.7479e+01,  1.6687e+02,  2.1870e+01,  9.3619e+01,  2.4110e+02,
         3.0015e+01,  1.7099e+02,  1.5358e+02,  2.1836e+02,  2.2045e+01,
         6.9627e+01,  4.4264e+01,  4.5123e+01,  6.4666e+01,  2.8267e+01,
         9.2330e+01,  1.1367e+02,  8.1579e+01,  5.3143e-02,  8.9557e+00,
         1.6787e+01,  5.3476e+01,  1.7080e+01,  1.4823e+02,  1.5794e+01,
         1.7169e+02,  1.1818e+02,  1.6461e+02,  2.9268e+01,  1.1645e+01,
         2.6790e+01,  7.3786e+01,  1.6184e+02,  6.9920e+01,  2.3911e+01,
         8.8144e+01,  9.3798e+01,  1.6240e+02,  4.4967e+01,  5.6550e+01,
         3.7613e+01, -2.4267e+00,  1.3699e+02,  2.4425e+01,  4.9086e+01,
         3.5311e+00,  1.1007e+02,  1.1787e+02,  2.0697e+02,  1.8106e+02,
         1.8731e+01,  4.4720e+01,  3.9023e+01,  1.6248e+02], device='cuda:0',
       grad_fn=<MaxBackward0>) tensor([1, 1, 1

tensor([ 67.3707,  43.7089, 112.4564,  39.3785, 186.3220, 190.3651,  65.2361,
         47.7097, 134.7629,  23.2277,  47.3535, 108.0532,  77.3974, 138.9057,
        209.4430, 165.5025,  17.9218,  -2.1813, 171.6750,  13.4030, 252.7088,
         78.9673,  14.0927, 118.0632,  65.9212,  29.3210,  30.6451,  78.5137,
         72.5763,  19.3669,  87.2036,  95.0379,  71.3021,  16.9306, 117.0201,
         53.8774,  27.3234, 132.1047, 152.5656, 163.8958,  40.0834, 265.6021,
         61.1969, 170.5714,  42.0102,  25.8975,  66.1372,   9.0398, 123.8287,
         27.5949,  48.8872, 204.4662, 162.6047,  48.7485,  73.4630,  35.4048,
         61.1690,  78.9100, 156.2849, 138.5040, 201.0883, 221.3802, 248.3111,
        135.3007], device='cuda:0', grad_fn=<MaxBackward0>) tensor([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0], device='cuda:0')
tens

...

tensor([ 5.0190, 27.4269,  6.7867, 36.3362, 30.6661, 31.2264, 19.3770, 10.4165,
         7.2213, 29.8963,  9.8873, 16.9908, 15.8355, 26.3528,  3.2470, 10.7742,
        36.1046, 16.5973, 17.4590, 47.5679, 12.1278, 33.5396,  2.7860, 30.5590,
         7.1426, 37.5624, 15.3671, 40.8060, 22.1018, 54.0853, 31.4767, 14.8470,
        20.9791, 19.6764, 39.7911, 23.3910, 25.8164,  6.9152, 16.7615, 40.8567,
        33.0266, 28.0213, 33.9597, 33.5122, 40.0984, 17.7759, 21.8765, 29.4707,
        23.1336, 23.8537, 29.2224, 27.7566,  3.9627, 15.5383,  8.3173, 32.0650,
        23.4029, 26.9368, 34.5159, 61.1881, 26.1112, 38.2846, 36.9410,  2.2070],
       device='cuda:0') tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
tensor([29.0426,  7.1622, 38.9299, 27.7799, 17.6876, 15.6265, 11.3957,  5.3930,
        31.8910,  2.8

tensor([38.7907, 49.4307, 45.4227, 20.3980, 35.4114, 22.5612, 18.0945, 12.0844,
        11.9593, 25.9591, 34.9649, 55.5441, 40.4547, 30.9336,  3.8610, 40.2614,
        17.9506,  3.4934, 17.3509, 29.3439, 23.1260, 23.2872, 42.6667, 25.2064,
         9.3859, 14.6936, 16.2157, 59.6262, 28.0781,  8.5922, 20.3373, 31.2668,
        28.9893, 13.3980, 34.3031, 30.8446,  0.9568, 33.0465,  2.1573, 29.1852,
        32.2253, 28.3362, 32.8281, 30.7283, 32.8950, 23.2275, 21.2547, 10.0330,
        26.7985, 15.4776,  9.7265, 31.2172, 25.7373, 19.7430, 37.6782, 16.5642,
        48.9548, 17.1567, 17.2532, 32.3415, 42.0677, 26.8626,  4.7005, 31.9222],
       device='cuda:0') tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], device='cuda:0')
tensor([28.6402,  0.4468, 25.4278, 23.5715, 17.0640,  1.1766, 38.5825, 36.9721,
        29.9571, 26.4

tensor([46.7658, 46.2161, 10.7785, 32.5918, 32.3164, 29.0620, 36.9119, 38.6811,
        25.0813, 12.1768, 10.9983,  0.8580, 13.3276, 18.5243, 22.4219, 14.9375,
        28.6686,  0.9213,  7.9907, 49.7283, 20.0269, 33.7028, 19.6401, 32.1263,
        16.9647, 37.1022, 28.0594, 11.2751, 29.6210, 54.0442, 32.8479, 30.5153,
         8.1839, 37.1084, 34.9043, 37.8775, 24.8318,  1.5560, 32.4073, 14.7455,
         8.5515, 26.0902, 15.5864, 21.4514, 24.7061,  4.5383, 22.0560, 19.0491,
         1.7087,  2.8976, 22.7183, 28.8809, 14.9621, 48.3180,  4.3235, 31.3489,
        30.9706, 36.7341, 33.8743, 20.9528, 24.5464, 16.3089, 12.4580,  5.7257],
       device='cuda:0') tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], device='cuda:0')
tensor([26.2549, 36.1630, 11.8252, 37.7627, 32.1719, 20.8705, 22.8455, 30.0494,
        25.2706,  2.1

tensor([30.6025, 16.7264, 41.3512, 32.8755, 35.5536, 30.6377,  2.8218, 19.3238,
        18.3228, 15.6950, 13.2872, 22.4362, 37.2666,  7.1224, 37.9187, 10.9086,
        23.2112,  1.9284, 27.7063, 15.5346, 21.7227, 20.7168, 25.3703, 50.9764,
        11.1529, 22.3044,  9.6804, 18.8004, 16.1012, 18.2736, 28.8714,  2.3441,
         3.2359, 10.9128, 47.4038, 36.1251, 59.1816, 23.2381, 34.0174, 25.1962,
        60.1456, 10.8593, 25.2729,  2.4631, 23.6253,  4.4519, 34.9543, 33.1063,
        25.6075, 34.9142, 52.5764, 27.9491, 63.9978, 25.3673, 41.3418, 18.4318,
        46.6897, 37.6244, 12.3058, 15.6772, 44.2612, 36.7300, 31.7167,  1.6891],
       device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
tensor([ 3.2004, 13.9054, 24.8817, 18.2798, 41.9937, 20.8813, 35.1594, 40.2746,
        34.4403,  6.3

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, chunks=9):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for chunk in range(chunks):
                X, Y = torch.load('data/images-' + str(chunk) + '.pt'),\
                       torch.load('data/labels-' + str(chunk) + '.pt')
                train = data_utils.TensorDataset(X, Y)
                train_loader = data_utils.DataLoader(train, batch_size=64, shuffle=True)
            
                for inputs, labels in train_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

            epoch_loss = running_loss / (chunks*2169)
            epoch_acc = running_corrects.double() / (chunks*2169)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [43]:
def test_model(model, criterion):
    model.eval()
    X, Y = torch.load('data/images-9.pt'),\
           torch.load('data/labels-9.pt')
    running_loss = 0.0
    running_corrects = 0
    train = data_utils.TensorDataset(X, Y)
    
    all_outputs = []
    all_labels = []
    
    train_loader = data_utils.DataLoader(train, batch_size=64, shuffle=True)
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        all_outputs.append(outputs.cpu())
        all_labels.append(labels.cpu())
            
    loss = running_loss
    acc = running_corrects.double() / (len(Y))

    print('Test Loss: {:.4f} Acc: {:.4f}'.format(
        loss, acc))
    
    all_outputs = torch.cat((*all_outputs,))
    all_labels = torch.cat((*all_labels,))
    
    return all_outputs, all_labels

In [74]:
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1,6]).to(device))
model = torch.load('models/vgg19_83.pt')
preds, labels = test_model(model, criterion)

Test Loss: 22600.3762 Acc: 0.8439


In [66]:
positive_preds = preds[:,1]
positive_preds

array([-12.109841, -17.844172, -40.22474 , ..., -56.390015, -59.2533  ,
       -28.091951], dtype=float32)

In [77]:
sm = nn.Softmax()
roc_preds = sm(preds).detach().numpy()[:,1]
roc_labels = labels.detach().numpy()
roc_preds, roc_labels

/home/paperspace/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


(array([1.0317825e-01, 0.0000000e+00, 9.6148248e-14, ..., 2.6629274e-10,
        6.2688561e-23, 6.6592953e-18], dtype=float32),
 array([0, 0, 0, ..., 0, 0, 0]))

In [78]:
def get_roc_curve(labels, preds):
    return roc_curve(labels, preds)
get_roc_curve(roc_labels, roc_preds)

(array([0.        , 0.05626598, 0.05984655, 0.06138107, 0.0629156 ,
        0.06445013, 0.06547315, 0.06803069, 0.06905371, 0.07109974,
        0.07109974, 0.07263427, 0.07263427, 0.07365729, 0.07365729,
        0.07468031, 0.07468031, 0.07519182, 0.07519182, 0.07672634,
        0.07672634, 0.07723785, 0.07723785, 0.08286445, 0.08286445,
        0.08337596, 0.08337596, 0.08797954, 0.08797954, 0.09002558,
        0.09002558, 0.09258312, 0.09258312, 0.09514066, 0.09514066,
        0.0971867 , 0.0971867 , 0.09769821, 0.09769821, 0.09974425,
        0.09974425, 0.10025575, 0.10025575, 0.10076726, 0.10076726,
        0.10332481, 0.10332481, 0.10485934, 0.10485934, 0.10588235,
        0.10588235, 0.11508951, 0.11508951, 0.11764706, 0.11764706,
        0.11815857, 0.11815857, 0.12071611, 0.12071611, 0.12429668,
        0.12429668, 0.12531969, 0.12531969, 0.12890026, 0.12890026,
        0.13043478, 0.13043478, 0.13401535, 0.13401535, 0.13554987,
        0.13554987, 0.1370844 , 0.1370844 , 0.13

In [16]:
a = torch.tensor([]).to(device)
b = torch.tensor([1,2,3]).to(device)

In [18]:
roc_curve

RuntimeError: Expected object of scalar type Float but got scalar type Long for sequence element 1 in sequence argument at position #1 'tensors'

TypeError: empty(): argument 'size' (position 1) must be tuple of ints, not NoneType

In [23]:
torch.empty(size=(2169,))

tensor([ 3.9236e-44,  4.5852e-41, -3.0092e+12,  ...,  5.3759e-24,
         4.5852e-41,  5.3757e-24])